# Train and deploy with Cloud AI Platform

## Create a training image
### Use Cloud Build with Kaniko

In [1]:
!gcloud config set builds/use_kaniko True

Updated property [builds/use_kaniko].


### Create Dockerfile

In [2]:
import os
import time

APP_FOLDER = '../training_app'
os.makedirs(APP_FOLDER, exist_ok=True)

In [3]:
%%writefile $APP_FOLDER/Dockerfile

FROM gcr.io/jk-sandbox12/ai_notebook_custom:latest
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Writing ../training_app/Dockerfile


### Create a training script

In [4]:
%%writefile $APP_FOLDER/train.py

import logging
import os
import subprocess
import sys

import fire
import joblib
import numpy as np
import pandas as pd

from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import Ridge
from sklearn.manifold import TSNE 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def select_model(X, y, n_features_options, l2_reg_options):
    
  # Set up grid search
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA()),
    ('regress', Ridge())
  ])

  param_grid = [
    {
        'reduce_dim': [PCA()],
        'reduce_dim__n_components': n_features_options,
        'regress': [Ridge()],
        'regress__alpha': l2_reg_options
    },
    {
        'reduce_dim': ['passthrough'],
        'regress': [PLSRegression(scale=False)],
        'regress__n_components': n_features_options
    }
  ]

  grid = GridSearchCV(pipeline, cv=10, n_jobs=None, param_grid=param_grid, scoring='neg_mean_squared_error', iid=False)

  grid.fit(X, y)

  return grid

def train(job_dir, data_path, n_features_options, l2_reg_options):
    
  # Load data from GCS
  df_train = pd.read_csv(data_path, index_col=0)
  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  # Find the best model
  y = df_train.octane
  X = df_train.drop('octane', axis=1)
  grid = select_model(X, y, n_features_options, l2_reg_options)

  logging.info("Best estimator: {}".format(grid.best_params_))
  logging.info("Best score: {}".format(grid.best_score_))
    
  # Retrain the best model on a full dataset
  best_estimator = grid.best_estimator_
  trained_pipeline = best_estimator.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=trained_pipeline, filename=model_filename)
  gcs_model_path = "{}/trained_model/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path))
  
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

Writing ../training_app/train.py


### Build the image

In [5]:
PROJECT_ID = !gcloud config list project --format "value(core.project)"
PROJECT_ID = PROJECT_ID[0]
IMAGE_REPO_NAME="training_app"
IMAGE_TAG="latest"
IMAGE_URI="gcr.io/{}/{}:{}".format(PROJECT_ID, IMAGE_REPO_NAME, IMAGE_TAG)

!gcloud builds submit --tag $IMAGE_URI $APP_FOLDER

Creating temporary tarball archive of 2 file(s) totalling 2.4 KiB before compression.
Uploading tarball of [../training_app] to [gs://jk-sandbox12_cloudbuild/source/1566143514.01-315a334aeb6746bc9f1e773e41c4dd35.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jk-sandbox12/builds/6378c3af-f828-48bd-a18b-ad36ba1f040b].
Logs are available at [https://console.cloud.google.com/gcr/builds/6378c3af-f828-48bd-a18b-ad36ba1f040b?project=532679469893].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "6378c3af-f828-48bd-a18b-ad36ba1f040b"

FETCHSOURCE
Fetching storage object: gs://jk-sandbox12_cloudbuild/source/1566143514.01-315a334aeb6746bc9f1e773e41c4dd35.tgz#1566143514947598
Copying gs://jk-sandbox12_cloudbuild/source/1566143514.01-315a334aeb6746bc9f1e773e41c4dd35.tgz#1566143514947598...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                          

## Submit a training job

In [12]:
JOB_NAME="JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
REGION="us-west1"
SCALE_TIER="BASIC"
JOB_DIR="gs://jk-demo-jobdir/{}".format(JOB_NAME)

DATA_PATH="gs://jk-demo-datasets/gasdata/gasdata.csv"
N_FEATURES_OPTIONS="[2,4,6]"
L2_REG_OPTIONS="[0.1,0.2,0.3,0.5]"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $JOB_DIR \
--master-image-uri $IMAGE_URI \
--scale-tier $SCALE_TIER \
-- \
--data_path $DATA_PATH \
--n_features_options $N_FEATURES_OPTIONS \
--l2_reg_options $L2_REG_OPTIONS

Job [JOB_20190818_150726] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20190818_150726

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20190818_150726
jobId: JOB_20190818_150726
state: QUEUED


In [13]:
!gcloud ai-platform jobs describe $JOB_NAME
!gcloud ai-platform jobs stream-logs $JOB_NAME

createTime: '2019-08-18T15:07:28Z'
etag: 3G1-3xv97no=
jobId: JOB_20190818_150726
state: PREPARING
trainingInput:
  args:
  - --data_path
  - gs://jk-demo-datasets/gasdata/gasdata.csv
  - --n_features_options
  - '[2,4,6]'
  - --l2_reg_options
  - '[0.1,0.2,0.3,0.5]'
  jobDir: gs://jk-demo-jobdir/JOB_20190818_150726
  masterConfig:
    imageUri: gcr.io/jk-sandbox12/training_app:latest
  region: us-west1
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/JOB_20190818_150726?project=jk-sandbox12

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2FJOB_20190818_150726&project=jk-sandbox12
INFO	2019-08-18 15:07:27 +0000	service		Validating job requirements...
INFO	2019-08-18 15:07:28 +0000	service		Job creation request has been successfully validated.
INFO	2019-08-18 15:07:28 +0000	service		Job JOB_20190818_150726 is queued.
INFO	2019-08-18 15:07:28 +0000	service		Waiting for job to be provisioned.
INFO	

## Create a training image with KFP SDK